In [1]:
import dask.dataframe as dd

In [2]:
# The task herein is to make a function that extracts rows with a call to local storag APIs

In [3]:
#When a site is visited for the first time, cookies are installed for subsequent visits. 
#A Web site might generate a unique ID number for each visitor and store the number on the machine of each user
#using a cookie file. Since these files are stored in the local storage of the machine, we want to know which sites
#made calls to localstorage hence the function below. 
# To start with I made a list of localstorage spaces likely to be employed for such purposes. 



In [4]:
# A list of localstorage spaces

stores = ['window.localStorage','window.document.cookie', 'window.sessionStorage']


In [5]:
# We start by defining a function we called 'findID'. Then we read the parquet file into a variable
#Second line involves filtering the calls made to ensure we get those within the list of localstorage spaces
# Last line is to make the table of results obtained. 

def findID(dataset):
    ddf = dd.read_parquet(dataset, engine='pyarrow') 
    unique_symbl = ddf.symbol.isin(stores) 
    new_table = ddf[unique_symbl] 
    return new_table
               
findID('./sample_10percent_value_1000_only.parquet').head() 

OSError: Passed non-file path: ./sample_10percent_value_1000_only.parquet

In [5]:
#If the function above is approved, I think the next step is to check for the string 'id/ID/userID' or something. 